# Install Packages

In [1]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.3.3.83 which is incompatible.
torch 2.5.1+cu124 requires 

# Import Libraries

In [ ]:
import evaluate
import pandas as pd
import numpy as np
import torch
import re
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

2025-04-25 02:52:46.596099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745549566.848184      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745549566.912841      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Set Device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Config

In [ ]:
MODEL_NAME = "microsoft/deberta-v3-base"
MAX_LENGTH = 512
NUM_LABELS = 8
TEST_SIZE = 0.1
VALID_SIZE = 0.1
LEARNING_RATE = 2e-5
BATCH_SIZE = 8
EPOCHS = 10
OUTPUT_DIR = "./math_classifier_results"
LOGGING_DIR = "./math_classifier_logs"

# Load Data

In [ ]:
df = pd.read_csv("/kaggle/input/classification-of-math-problems-by-kasut-academy/train.csv")

print(f"Loaded DataFrame shape: {df.shape}")
print("Label distribution:\n", df['label'].value_counts())

# Data Preprocessing

In [ ]:
def clean_math_text_final(text):
    
    text = str(text)
    text = re.sub(r'^\s*\d+\.\s*', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'#\w+', ' ', text)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r' ', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()

    return text

print("\n--- Applying Text Cleaning ---")
df['cleaned_question'] = df['Question'].apply(clean_math_text_final)
print("Cleaning done.")

# Prepare Data for Training

In [ ]:
print("\n--- Creating Dataset & Casting Label Type ---")
dataset = Dataset.from_pandas(df)

class_label_feature = ClassLabel(num_classes=NUM_LABELS)
dataset = dataset.cast_column('label', class_label_feature)

print(f"Dataset features after casting 'label' column:")
print(dataset.features)

print("\n--- Splitting Dataset ---")
train_test_split = dataset.train_test_split(test_size=TEST_SIZE, stratify_by_column='label')
train_valid_split = train_test_split['train'].train_test_split(test_size=VALID_SIZE / (1 - TEST_SIZE), stratify_by_column='label')

final_datasets = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test'],
    'test': train_test_split['test']
})
print("Dataset splits created:")
print(final_datasets)

# Tokenization

In [ ]:
print("\n--- Tokenization ---")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    
    return tokenizer(examples["cleaned_question"],
                     padding="max_length",
                     truncation=True,
                     max_length=MAX_LENGTH)

tokenized_datasets = final_datasets.map(tokenize_function, batched=True)


tokenized_datasets = tokenized_datasets.remove_columns(["Question", "cleaned_question"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
print("Tokenization complete.")
print("Example train sample:", tokenized_datasets["train"][0])

# Define Evaluation Metrics

In [ ]:
print("\n--- Setting up Metrics ---")
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1_weighted = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    f1_micro = f1_metric.compute(predictions=predictions, references=labels, average="micro")["f1"]
    return {"accuracy": accuracy, "f1_weighted": f1_weighted, "f1_micro": f1_micro}

# Load and Train deberta-v3 model with Trainer

In [ ]:
print("\n--- Loading Model ---")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
)

model.to(device)
print(f"Model '{MODEL_NAME}' loaded for {NUM_LABELS}-class classification.")


print("\n--- Defining Training Arguments ---")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=LOGGING_DIR,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,
    push_to_hub=False,
    report_to="none",
    fp16=True,
    save_total_limit=3
)

print("\n--- Initializing Trainer ---")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


print("\n--- Starting Fine-Tuning ---")
train_result = trainer.train()


trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()


trainer.save_model(OUTPUT_DIR + "/best_model")
print("Training finished. Best model saved.")


print("\n--- Evaluating on Test Set ---")
test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])

print("Test Set Evaluation Results:")
print(test_results)
trainer.log_metrics("eval", test_results)
trainer.save_metrics("eval", test_results)

Using device: cuda
Loaded DataFrame shape: (10189, 2)
Label distribution:
 label
0    2618
1    2439
5    1827
4    1712
2    1039
3     368
6     100
7      86
Name: count, dtype: int64

--- Applying Text Cleaning ---
Cleaning applied.

--- Creating Dataset & Casting Label Type ---


Casting the dataset:   0%|          | 0/10189 [00:00<?, ? examples/s]

Dataset features after casting 'label' column:
{'Question': Value(dtype='string', id=None), 'label': ClassLabel(names=['0', '1', '2', '3', '4', '5', '6', '7'], id=None), 'cleaned_question': Value(dtype='string', id=None)}

--- Splitting Dataset ---
Dataset splits created:
DatasetDict({
    train: Dataset({
        features: ['Question', 'label', 'cleaned_question'],
        num_rows: 8151
    })
    validation: Dataset({
        features: ['Question', 'label', 'cleaned_question'],
        num_rows: 1019
    })
    test: Dataset({
        features: ['Question', 'label', 'cleaned_question'],
        num_rows: 1019
    })
})

--- Tokenization ---


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Micro
1,1.026100,0.649270,0.788027,0.776827,0.788027
2,0.586200,0.559017,0.821394,0.818758,0.821394
3,0.445200,0.589507,0.823356,0.819444,0.823356
4,0.335100,0.625525,0.825319,0.823427,0.825319
5,0.275600,0.682424,0.833170,0.831926,0.833170
6,0.223000,0.788331,0.826300,0.826141,0.826300
7,0.203600,1.175128,0.821394,0.821209,0.821394
8,0.183200,1.299756,0.832188,0.830496,0.832188
9,0.175800,1.542676,0.837095,0.836012,0.837095
10,0.155300,1.657266,0.840039,0.838504,0.840039


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

***** train metrics *****
  epoch                    =       10.0
  total_flos               = 19974747GF
  train_loss               =     0.3609
  train_runtime            = 1:52:28.55
  train_samples_per_second =     12.078
  train_steps_per_second   =      0.756
Training finished. Best model saved.

--- Evaluating on Test Set ---


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Test Set Evaluation Results:
{'eval_loss': 1.6139366626739502, 'eval_accuracy': 0.8439646712463199, 'eval_f1_weighted': 0.8410343516181031, 'eval_f1_micro': 0.8439646712463199, 'eval_runtime': 29.979, 'eval_samples_per_second': 33.99, 'eval_steps_per_second': 2.135, 'epoch': 10.0}
***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =      0.844
  eval_f1_micro           =      0.844
  eval_f1_weighted        =      0.841
  eval_loss               =     1.6139
  eval_runtime            = 0:00:29.97
  eval_samples_per_second =      33.99
  eval_steps_per_second   =      2.135


# Run predictions on Test set and Submit

In [ ]:
print("\n--- Preparing Competition Test Set ---")

comp_test_df = pd.read_csv("/kaggle/input/classification-of-math-problems-by-kasut-academy/test.csv")
print(f"Loaded competition test data shape: {comp_test_df.shape}")

print("Cleaning competition test data...")
comp_test_df['cleaned_question'] = comp_test_df['Question'].apply(clean_math_text_final)
print("Cleaning complete.")

predict_dataset = Dataset.from_pandas(comp_test_df[['cleaned_question']])
print("Competition test data converted to Dataset format.")
print(predict_dataset)

def tokenize_for_predict(examples):
    return tokenizer(examples["cleaned_question"],
                     padding="max_length",
                     truncation=True,
                     max_length=MAX_LENGTH)

print("\n--- Tokenizing Competition Test Set ---")
tokenized_predict_dataset = predict_dataset.map(tokenize_for_predict, batched=True)

tokenized_predict_dataset = tokenized_predict_dataset.remove_columns(["cleaned_question"])
tokenized_predict_dataset.set_format("torch")
print("Tokenization complete.")
print("Example prediction sample:", tokenized_predict_dataset[0])


print("\n--- Making Predictions ---")
predictions_output = trainer.predict(tokenized_predict_dataset)

logits = predictions_output.predictions

predicted_labels = np.argmax(logits, axis=-1)
print("Predictions generated.")
print("Example predicted labels:", predicted_labels[:10])

print("\n--- Creating Submission File ---")
submission_df = pd.DataFrame({
    'id': comp_test_df['id'],
    'label': predicted_labels
})

submission_filename = 'submission.csv'
submission_df.to_csv(submission_filename, index=False)
print(f"Submission file '{submission_filename}' created successfully.")
print(submission_df.head())


--- Preparing Competition Test Set ---
Loaded competition test data shape: (3044, 2)
Cleaning competition test data...
Cleaning complete.
Competition test data converted to Dataset format.
Dataset({
    features: ['cleaned_question'],
    num_rows: 3044
})

--- Tokenizing Competition Test Set ---


Map:   0%|          | 0/3044 [00:00<?, ? examples/s]

Tokenization complete.
Example prediction sample: {'input_ids': tensor([    1,  2165,   280, 51945,   767,  1842,   341,   667,   341, 10015,
         1225,   667,   341,   706,  8454,  1225,   667,   341,  4604, 17334,
          270,   584,   260,  2620,   673,   280,     2,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
